In [2]:
import pandas as pd
import numpy as np
import scipy
import re
from sklearn.decomposition import PCA
import fancyimpute as fi
import math

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
most_matches = pd.read_csv('most_seasons_unnormalized.csv')
recent_matches = pd.read_csv('recent_seasons_unnormalized.csv')

most_matches = most_matches.fillna(np.nan)
most_matches = most_matches.replace(-1, np.nan)

recent_matches = recent_matches.fillna(np.nan)
recent_matches = recent_matches.replace(-1, np.nan)

74


In [3]:
most_matches.drop(list(most_matches.filter(regex='Unnamed: ')), axis=1, inplace=True)
recent_matches.drop(list(recent_matches.filter(regex='Unnamed: ')), axis=1, inplace=True)

all_matches = most_matches.append(recent_matches, ignore_index=True)
all_matches.drop(['__home_team_fifa_api_id', '__away_team_fifa_api_id'], axis=1, inplace=True)

In [4]:
## Fill missing betting odds

betting_odds_home = ['B365H', 'BWH', 'IWH', 'LBH', 'PSH', 'WHH', 'SJH', 'VCH', 'GBH', 'BSH']
betting_odds_draw = ['B365D', 'BWD', 'IWD', 'LBD', 'PSD', 'WHD', 'SJD', 'VCD', 'GBD', 'BSD']
betting_odds_away = ['B365A', 'BWA', 'IWA', 'LBA', 'PSA', 'WHA', 'SJA', 'VCA', 'GBA', 'BSA']
betting_odds_all = [betting_odds_home, betting_odds_draw, betting_odds_away]
betting_odds_combined = betting_odds_home + betting_odds_draw + betting_odds_away

rows = all_matches[pd.isnull(all_matches[betting_odds_combined]).any(axis=1)]

for index, row in rows.iterrows():
    if index % 1000 == 0:
        print(index)
    for betting_odds in betting_odds_all:
        mean = np.mean(row[betting_odds])
        all_matches.loc[index, row[betting_odds].index[row[betting_odds].isnull().tolist()]] = mean

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000


In [5]:
## Fill missing team attributes

home = all_matches.filter(regex='__home_')
away = all_matches.filter(regex='__away_')

home_filled = fi.KNN().fit_transform(home)
home_filled = pd.DataFrame(data=home_filled, columns=home.columns, index=home.index)

away_filled = fi.KNN().fit_transform(away)
away_filled = pd.DataFrame(data=away_filled, columns=away.columns, index=away.index)

all_matches_filled = all_matches.copy()
all_matches_filled[home.columns] = home_filled
all_matches_filled[away.columns] = away_filled

assert(not all_matches_filled.isnull().values.any())

Imputing row 1/16576 with 1 missing, elapsed time: 52.412
Imputing row 101/16576 with 1 missing, elapsed time: 52.441
Imputing row 201/16576 with 1 missing, elapsed time: 52.460
Imputing row 301/16576 with 1 missing, elapsed time: 52.475
Imputing row 401/16576 with 1 missing, elapsed time: 52.494
Imputing row 501/16576 with 1 missing, elapsed time: 52.508
Imputing row 601/16576 with 1 missing, elapsed time: 52.523
Imputing row 701/16576 with 0 missing, elapsed time: 52.536
Imputing row 801/16576 with 0 missing, elapsed time: 52.536
Imputing row 901/16576 with 0 missing, elapsed time: 52.537
Imputing row 1001/16576 with 0 missing, elapsed time: 52.537
Imputing row 1101/16576 with 1 missing, elapsed time: 52.549
Imputing row 1201/16576 with 1 missing, elapsed time: 52.568
Imputing row 1301/16576 with 1 missing, elapsed time: 52.587
Imputing row 1401/16576 with 1 missing, elapsed time: 52.631
Imputing row 1501/16576 with 1 missing, elapsed time: 52.655
Imputing row 1601/16576 with 1 missi

Imputing row 14301/16576 with 1 missing, elapsed time: 54.386
Imputing row 14401/16576 with 1 missing, elapsed time: 54.444
Imputing row 14501/16576 with 1 missing, elapsed time: 54.512
Imputing row 14601/16576 with 1 missing, elapsed time: 54.570
Imputing row 14701/16576 with 1 missing, elapsed time: 54.637
Imputing row 14801/16576 with 1 missing, elapsed time: 54.697
Imputing row 14901/16576 with 1 missing, elapsed time: 54.759
Imputing row 15001/16576 with 1 missing, elapsed time: 54.826
Imputing row 15101/16576 with 1 missing, elapsed time: 54.917
Imputing row 15201/16576 with 1 missing, elapsed time: 54.944
Imputing row 15301/16576 with 0 missing, elapsed time: 55.049
Imputing row 15401/16576 with 0 missing, elapsed time: 55.049
Imputing row 15501/16576 with 0 missing, elapsed time: 55.050
Imputing row 15601/16576 with 0 missing, elapsed time: 55.050
Imputing row 15701/16576 with 0 missing, elapsed time: 55.051
Imputing row 15801/16576 with 0 missing, elapsed time: 55.051
Imputing

Imputing row 12701/16576 with 1 missing, elapsed time: 50.971
Imputing row 12801/16576 with 1 missing, elapsed time: 50.991
Imputing row 12901/16576 with 1 missing, elapsed time: 51.013
Imputing row 13001/16576 with 1 missing, elapsed time: 51.037
Imputing row 13101/16576 with 1 missing, elapsed time: 51.065
Imputing row 13201/16576 with 1 missing, elapsed time: 51.092
Imputing row 13301/16576 with 1 missing, elapsed time: 51.113
Imputing row 13401/16576 with 1 missing, elapsed time: 51.136
Imputing row 13501/16576 with 0 missing, elapsed time: 51.144
Imputing row 13601/16576 with 0 missing, elapsed time: 51.144
Imputing row 13701/16576 with 0 missing, elapsed time: 51.144
Imputing row 13801/16576 with 1 missing, elapsed time: 51.151
Imputing row 13901/16576 with 1 missing, elapsed time: 51.174
Imputing row 14001/16576 with 1 missing, elapsed time: 51.196
Imputing row 14101/16576 with 1 missing, elapsed time: 51.220
Imputing row 14201/16576 with 1 missing, elapsed time: 51.242
Imputing

In [6]:
all_matches_normed = all_matches_filled.copy()
for col in all_matches.columns:
    all_matches_normed[col] = all_matches_filled[col] - np.mean(all_matches_filled[col])
    all_matches_normed[col] /= np.std(all_matches_normed[col])
    
all_matches_normed

,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,...,__away_chanceCreationShooting,__away_chanceCreationShootingClass,__away_chanceCreationPositioningClass,__away_defencePressure,__away_defencePressureClass,__away_defenceAggression,__away_defenceAggressionClass,__away_defenceTeamWidth,__away_defenceTeamWidthClass,__away_defenceDefenderLineClass
0,-0.767281,1.742910,2.083575,-0.815658,1.529543,1.726091,-0.859861,1.556971,1.930649,-0.812463,...,-0.368113,0.273463,0.407242,0.314847,-0.426173,0.015182,-0.413472,-0.214925,0.096447,0.351089
1,-0.092533,-0.532952,-0.542277,-0.001090,-0.452932,-0.592994,0.061378,-0.649624,-0.631603,-0.106502,...,-0.836390,0.273463,0.407242,1.251411,-0.426173,2.020471,2.796697,1.810373,-2.659950,0.351089
2,0.107783,-0.491572,-0.630647,0.086185,-0.452932,-0.635940,-0.070227,-0.649624,-0.565051,0.065684,...,0.568442,0.273463,0.407242,-0.153435,-0.426173,0.015182,-0.413472,-1.227574,0.096447,0.351089
3,-0.197963,-0.491572,-0.471581,-0.321099,-0.497988,-0.378264,-0.333438,-0.533488,-0.365395,-0.221292,...,-0.368113,0.273463,0.407242,2.187975,3.310536,1.017827,-0.413472,1.810373,-2.659950,0.351089
4,1.620694,-0.243297,-0.802338,1.686229,-0.272707,-0.836354,1.640645,0.047195,-0.897811,1.672749,...,0.568442,0.273463,0.407242,2.187975,3.310536,0.015182,-0.413472,1.810373,-2.659950,0.351089
5,-0.197963,-0.491572,-0.491780,-0.175641,-0.497988,-0.507102,-0.201833,-0.649624,-0.465223,-0.192595,...,0.100164,0.273463,0.407242,2.187975,3.310536,2.020471,2.796697,1.810373,-2.659950,0.351089
6,0.118325,-0.491572,-0.630647,-0.030182,-0.452932,-0.578678,-0.136030,-0.649624,-0.531775,0.008288,...,-0.368113,0.273463,0.407242,2.187975,3.310536,2.020471,2.796697,1.810373,-2.659950,0.351089
7,-0.092533,-0.532952,-0.542277,-0.088366,-0.588100,-0.521417,-0.136030,-0.649624,-0.531775,-0.106502,...,0.568442,0.273463,0.407242,2.187975,3.310536,0.015182,-0.413472,1.810373,-2.659950,0.351089
8,-0.619680,-0.119159,0.442418,-0.641108,-0.137538,0.523603,-0.695354,0.163332,0.399953,-0.623059,...,0.568442,0.273463,0.407242,2.187975,3.310536,1.519149,-0.413472,1.810373,-2.659950,0.351089
9,-0.092533,-0.408814,-0.517029,-0.175641,-0.588100,-0.478471,-0.136030,-0.649624,-0.531775,-0.192595,...,-0.368113,0.273463,0.407242,2.187975,3.310536,1.017827,-0.413472,1.810373,-2.659950,0.351089


In [7]:
## n_components == 'mle' and svd_solver == 'full'
##    Uses Minka's MLE to guess the dimension of the data and select the components to keep

explained_variance_ratios = [0.75, 0.85, 0.9, 0.95, 0.99]

for explained_variance_ratio in explained_variance_ratios:
    pca = PCA(n_components=explained_variance_ratio, svd_solver='full')
    fit = pca.fit(all_matches_normed)
    
    print('n_components:', fit.n_components_)
    print('percent variance:', np.sum(fit.explained_variance_ratio_))

    all_matches_pca = pca.fit_transform(all_matches_normed)
    all_matches_pca = pd.DataFrame(all_matches_pca)
    
    most_matches_pca = all_matches_pca[:len(most_matches)]
    recent_matches_pca = all_matches_pca[-len(recent_matches):]
    
    pct = int(explained_variance_ratio * 100)
    most_matches_pca.to_csv('most_seasons_PCA_{}_pct_{}_components.csv'.format(pct, fit.n_components_), index=False)
    recent_matches_pca.to_csv('recent_seasons_PCA_{}_pct_{}_components.csv'.format(pct, fit.n_components_), index=False)

n_components: 13
percent variance: 0.750343082193955
n_components: 21
percent variance: 0.8505412173974698
n_components: 27
percent variance: 0.9054724788671208
n_components: 34
percent variance: 0.9518719703203508
n_components: 44
percent variance: 0.9907023075476148
